In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models
from torchvision.datasets import ImageNet
from torchvision import transforms
import os
import sys
sys.path.append("/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/src")
from sparsity_in_silico.sparsity_CNN import response_prob
#import imageio
import numpy as np
import h5py

In [2]:
os.getcwd()

'/Users/tizianocausin/Desktop/backUp20240609/summer2025/ponce_lab/exp_set/python_scripts/scripts'

In [3]:
#alexnet = models.alexnet(pretrained=True).eval()
alexnet = models.alexnet(weights=True).eval()
preprocess = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.3806, 0.4242, 0.3794], std=[0.2447, 0.2732, 0.2561]
        ),  # Normalization for pretrained model
    ]
)

#reader = imageio.get_reader(path2vid)
conv_layers = [
    "conv_layer1",
    "conv_layer4",
    "conv_layer7",
    "conv_layer9",
    "conv_layer11",
]
conv_layers_idx = [1, 4, 7, 9, 11]

fc_layers = ["fc_layer2", "fc_layer5"]
fc_layers_idx = [2, 5]

/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
output_len_conv = [193600, 139968, 64896, 43264, 43264]
rand_idx_conv = []
for len_repr in output_len_conv:
    rand_idx_conv.append(
        np.random.choice(np.arange(len_repr - 1), size=len_repr // 50, replace=False)
    )

output_len_fc = [4096, 4096]
rand_idx_fc = []
for len_repr in output_len_fc:
    rand_idx_fc.append(
        np.random.choice(np.arange(len_repr - 1), size=len_repr // 50, replace=False)
            )

# %%
def wrapper_hook(layer, rand_idx):
    def hook_func(module, input, output):
        out = output.detach().half().reshape(-1)
        out = out[rand_idx]
        feats[layer].append(
            out
        )  # half makes it become float16, reshape(-1) vectorizes it

    return hook_func


hook_handle = []
for conv_idx in range(len(conv_layers_idx)):
    hook_handle.append(
        alexnet.features[conv_layers_idx[conv_idx]].register_forward_hook(
            wrapper_hook(conv_layers[conv_idx], rand_idx_conv[conv_idx])
        )
    )


for fc_idx in range(len(fc_layers_idx)):
    hook_handle.append(
        alexnet.classifier[fc_layers_idx[fc_idx]].register_forward_hook(
            wrapper_hook(fc_layers[fc_idx], rand_idx_fc[fc_idx])
        )
    )


In [5]:
# to remove handles
#for h in hook_handle:
#    h.remove()
#hook_handle.clear()

In [6]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
val_set = ImageNet(root="/Users/tizianocausin/Library/CloudStorage/OneDrive-SISSA/data_repo/exp_set_data/imagenet", split="val", transform=transform)

In [7]:

val_loader = DataLoader(
    val_set,
    batch_size=1,         
    shuffle=False,         
    num_workers=1
)

In [8]:
feats = {
        "conv_layer1": [],
        "conv_layer4": [],
        "conv_layer7": [],
        "conv_layer9": [],
        "conv_layer11": [],
        "fc_layer2": [],
        "fc_layer5": [],
    }
with torch.no_grad():  # Disable gradient tracking for evaluation
    for inputs, labels in val_loader:
    
        outputs = alexnet(inputs)           # Forward pass
        _, preds = torch.max(outputs, 1)  # Get predicted class indices

        # Example: print first batch predictions
        print(preds)
        break  # Remove this to go through the whole dataset

tensor([395])


In [9]:
response_prob(feats)

conv_layer1
torch.Size([3872])
conv_layer4
torch.Size([2799])
conv_layer7
torch.Size([1297])
conv_layer9
torch.Size([865])
conv_layer11
torch.Size([865])
fc_layer2
torch.Size([81])
fc_layer5
torch.Size([81])


{'conv_layer1': tensor([0., 0., 1.,  ..., 0., 0., 1.]),
 'conv_layer4': tensor([0., 0., 0.,  ..., 1., 0., 0.]),
 'conv_layer7': tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 'conv_layer9': tensor([0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1.,
         0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0.,
         0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
         0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
       